# Modules

This notebook aims to generate outputs for all callable modules within this package, note the speed for 1-2 chosen examples and plot information that are returned. 

In [1]:
import hazards
import time
import matplotlib.pyplot as plt
from shapely.geometry import LineString, Point, Polygon, MultiLineString
import numpy as np

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#Peebles Example 
north = 55.6658  # North latitude
south = 55.6372  # South latitude
east = -3.1651   # East longitude
west = -3.2116   # West longitude

buffer = 0.005

#Exeter
name = 'Exeter'

geotiff_file = 'Peebles-30m-DEM.tif'

## River Modules

### .river_list

This returns a list of the river, their locations and and OSMid. This also returns the polygon associated with the chosen region. 
If a buffer is specified this will also return the buffered polygon as well 

In [ ]:
#st = time.perf_counter()
river, polygon, buffered_polygon = hazards.river_list('ខេត្តកំពង់ចាម', buffer = buffer)
#river, polygon, buffered_polygon = hazards.river_list(name, buffer=buffer)
#et = time.perf_counter()
#print(f'Time taken={et-st}seconds')

In [ ]:
river, polygon, buffered_polygon = hazards.river_list('Banjarmasin', buffer = buffer)

In [ ]:
fig, ax = plt.subplots(figsize =(10,10))

for geom in river['new geometry']:
    if isinstance(geom, MultiLineString):
        # If it's a MultiLineString, use the `geoms` property to iterate through LineStrings
        for line in geom.geoms:
            x, y = line.coords.xy
            ax.plot(x, y, color='tab:blue')
    elif isinstance(geom, LineString):
        # If it's a LineString, plot it directly
        x, y = geom.coords.xy
        ax.plot(x, y, color='tab:blue')

# Plot the polygon

x, y = polygon.exterior.xy
ax.fill(x, y, alpha=0.1, color='green')

x, y = buffered_polygon.exterior.xy
ax.plot(x, y, alpha = 0.1, color = 'green', linestyle = '-')

#plt.title(f'Polygon of {name}, and rivers contained within {buffer} degree buffer of polygon')
plt.title(f'Polygon of region, and rivers contained within {buffer} degree buffer of polygon')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(linestyle = '--')
plt.gca().set_aspect('equal')

# Show the plot
plt.savefig('river_list.png')
plt.show()

To switch between the river plot for Peebles and Exeter and any other location, the lines in the above box can be toggled or the north, south, east, west and name values can be changed above. 

Examples to try: 
- Peterborough (England)
- 경안동 (South Korea)
- 官渡区 (China)
- Maseru District (Lesotho)
- Panajachel (Guatemala)
- Партызанскі раён (Belarus)
- ខេត្តកំពង់ចាម (Cambodia) 

### WARNING: there are limits to the size that the location can be. Too large will crash this notebook. The region for the river information can be larger than the buildings, however there is still a limit to the area size

==================================================================================================================
### .river_elevation

This returns the river list file containing the geometry of each point, the name of each river section, new geometries after the buffer has been applied and the elevation of each point of the river. 

This will only work for regions within the given geotiff region (approximately Scottish bounders)

In [ ]:
st = time.perf_counter()
river = hazards.river_elevation(geotiff_file, north, south, east, west, buffer = buffer)

et = time.perf_counter()
print(f'Time taken={et-st}seconds')

In [ ]:
river

Examples to try: 
- Galashiels
- Midlothian
- West Lothian
- Scottish Borders
- lat lon box within the given region


=========================================================================== 
----

## Building Modules

Building areas have a smaller limit to the maximum size as it contains more information per area in urban regions. 

### .building_plot

In [ ]:
st = time.perf_counter()
hazards.building_plot(name)
#hazards.building_plot(north, south, east, west)

et = time.perf_counter()
print(f'Time taken={et-st}seconds')

This returns of plot of the buildings within the chosen area. This helps to show the size and distribution of the buildings but is not in and of itself used for analysis. However it is a helpful visual for a chosen area, as well as a quick way to determine how well Open Street Maps has mapped the region.

Examples to try: 
- 경안동 (South Korea)
- 官渡区 (China)
- Peterborough (England)
- ⴱⴻⵔⴵ ⵍⴱⴻⵃⵔⵉ (Algeria)
- Batas Dusun Lendang Ara 3 (Indonesia)

====================================================================================================================
### .building_pieplot

This shows the pie chart distribution of the building types in a given area. The number of building types within the piechart and barchart can be specified but they are automatically set to 9 and 20.

In [ ]:
st = time.perf_counter()
hazards.building_pieplot('官渡区')

et = time.perf_counter()
print(f'Time taken={et-st}seconds')

Some regions do not work due to not having buildings specified as 'type' (경안동, Batas Dusun Lendang Ara 3, etc.)

This relates to the information within the OSM system. South Korea for example is known to limit the information on buildings that is held by companies outside of the country. Google maps for example has poor information on South Korean building types in comparison to the countries more commonly used Naver maps. Things like this show a limitation of the OSM data and are important to be aware of through the use of this package. 

====================================================================================================================
### .building_treemap

Treemaps in this package show the same information as the above pie chart, however show it in a more interpretable way. 

In [3]:
#st = time.perf_counter()
hazards.building_treemap(north, south, east, west)

#et = time.perf_counter()
#print(f'Time taken={et-st}seconds')

AttributeError: module 'hazards' has no attribute 'building_treemap'

====================================================================================================================
### .building_buffer

This returns the buffer of the region (this can be simply a buffer from the river or a cut buffer between two distances). It then plots the same piechart as above, again which alterable numbers of catergories in each plot

In [ ]:
st = time.perf_counter()
b = hazards.buffered_buildings('Партызанскі раён', inner_distance= 0.002, buffer_distance= 0.01, m=10)                #, buffer_distance= None, inner_distance=0.003) <-- INNER ADDS SEC LIST

et = time.perf_counter()
print(f'Time taken={et-st}seconds')

====================================================================================================================
### .building_elev

This returns a similar result to .river_elevation, however with the building absolute elevation as well as the buildings relative elevation to the closest river point. Again, this can only be completed for regions within the DEM dataset. Here it is the Scottish Borders. 

In [ ]:
buildings = hazards.building_elev(geotiff_file, north, south, east, west)

In [ ]:
buildings

In [ ]:
#Plot
colourmap = np.arange(buildings['relative elevation'].min(), buildings['relative elevation'].max(), 1)
cmap = plt.get_cmap('rainbow_r')

fig, ax = plt.subplots(figsize=(10, 10))

buildings.plot(column='relative elevation', cmap=cmap, ax=ax)

# Set axis labels and title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Buildings Colored by Elevation')

sm = plt.cm.ScalarMappable(cmap=cmap)
sm.set_array(colourmap)  
cbar = plt.colorbar(sm, ax=ax, label='relative Elevation to River /m')


# Display the plot
plt.savefig('building_elevation_graph.png')
plt.show()

Examples to try:
- Galashiels
- south = 55.5395, west = -2.864486, north = 55.5595, east = -2.820198 (Selkirk)

====================================================================================================================
.building_dis

In [ ]:
build = hazards.building_dis('官渡区')

========================================================================== 
----



## Earthquake Module

### .earthquake_buffer

This plots the distance buffers away from a chosen lat lon location of the epicentre of a simple earthquake. If no aregument is given for the location it is set to the middle of the region given. 

In [ ]:
hazards.earthquake_buffer(north, south, east, west)